In [ ]:
import os

# import cv2
import requests
import json
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager
# from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import ElementClickInterceptedException

import pandas as pd
import pyautogui
import time

from PIL import Image
from PIL.ExifTags import TAGS

import re
data_folder = '../../../src/components/BlogPosts/BerkeleyNature/Data'

In [ ]:
images_folder = '../../../src/components/BlogPosts/BerkeleyNature/Images'
absolute_path = r"C:\Users\tyler\PersonalProjects\small-data-blog\src\components\BlogPosts\BerkeleyNature\Images"
images = os.listdir(images_folder)
extensions = ['.jpg','.JPG']
images = [os.path.join(absolute_path,i) for i in images if any(ext in i for ext in extensions)]
images[:1]
# 'src\components\BlogPosts\BerkeleyNature\Images'

sort images by date (oldest --> newest)

In [ ]:
len(images)

In [ ]:
image_date_dict = {}
r = re.compile('202.*')
for img in images:
    img_name = img.split('\\')[-1]
    # print(img_name)
    image = Image.open(img)
    exifdata = image.getexif()
    tag_values = [e for e in list(exifdata.values()) if isinstance(e, str)]
    if len(tag_values) == 0:
        image_date_dict[img_name] = None
    # print(tag_values)
    else:
        try:
            date = list(filter(r.match, tag_values))[0]
        except:
            print(img_name,date)
            date = '2022/12/14'
        date = date.split(' ')[0].replace(':','/')
        # print(date)
        image_date_dict[img_name] = date
image_date_dict

In [ ]:
len(image_date_dict)

In [ ]:
# sort images by value (date), moving null dates to end
sorted_images_dict = {k: v for k, v in sorted(image_date_dict.items(), key=lambda x: (x[1] is None, x[1]))}
len(sorted_images_dict)

In [ ]:
# remove null dates
sorted_images_no_null = [k for k,v in sorted_images_dict.items() if v is not None]
print(len(sorted_images_no_null))
sorted_images = sorted_images_dict.keys()

In [ ]:
# NEW : get image tags via API
API_ENDPOINT = "https://imagerecognize.com/api/v3/"
f = open('../api_keys.json')
key = json.load(f)
API_KEY = key["ImageRecognize"]

In [ ]:
excel_ordered_images = pd.read_csv(data_folder + '/image_metadata.csv').loc[:,'img']
excel_ordered_images

In [ ]:
def reduce_image_size(image):
    load_image = Image.open(os.path.join(images_folder,image))
    load_image = load_image.resize((int(load_image.size[0]/2),int(load_image.size[1]/2)),Image.ANTIALIAS)
    load_image.save(os.path.join(images_folder,'small_folder','smaller_'+image), optimize=True, quality=85)

In [ ]:
offset = 124
image_json = {}

headers = {'Accept': 'application/json'}

for i, image in enumerate(excel_ordered_images[offset:]):
    time.sleep(1)
    print(i,'/',len(excel_ordered_images) - offset,image)
    if i % 5 == 0:
        
        print('json:',image_json)
    with open(os.path.join(images_folder,image) , 'rb') as f:
        f = {'file': f.read()}

    d = {'apikey': API_KEY,
        'type': 'objects',
        'max_labels': 10,
        'min_confidence': 80}

    r = requests.post(url=API_ENDPOINT, data=d, files=f,headers=headers)
    if type(r) == requests.models.Response:
        print('type class')
        if str(r) =='<Response [403]>':
            print('error 413')
            # if image too large, save as smaller size and retest
            reduce_image_size(image)
            with open(os.path.join(images_folder,'small_folder','smaller_'+image) , 'rb') as f:
                f = {'file': f.read()}
            r = requests.post(url=API_ENDPOINT, data=d, files=f)
            result = json.loads(r.text)
            image_json[image] = result.get('data')
            continue
        else:
            result = json.loads(r.text)
            image_json[image] = result.get('data')
            continue

    else:
        print('other type',type(r))
        if result['status'] == 200:
            image_json[image] = result.get('data')
        elif result['status'] == 413:
            print('error 413')
            # if image too large, save as smaller size and retest
            reduce_image_size(image)
            with open(os.path.join(images_folder,'smaller_',image) , 'rb') as f:
                f = {'file': f.read()}
            r = requests.post(url=API_ENDPOINT, data=d, files=f)
            result = json.loads(r.text)
            image_json[image] = result.get('data')
            continue
        else:
            print(offset + i, image,'error:',result)
image_json

In [ ]:
len(image_json)
# i
# Image size too large
# sub_list = [i for i in excel_ordered_images[:124] if i not in ['marina (4).jpg','tree (2).jpg']]
# len(sub_list)
# img_dict = dict(zip(sub_list,image_json))
# img_dict

In [ ]:
with open(data_folder + '/image_tags1.json', 'w') as outfile:
    json.dump(image_json, outfile)

In [52]:
# combine data
f0 = open(data_folder + '/image_tags0.json', 'r')
data0 = json.load(f0)

f1 = open(data_folder + '/image_metadata.json', 'r')
data1 = json.load(f1)

for d in data1:
    # print(d)
    try:
        d['tags'] = [o['name'] for o in data0.get(d.get('img')).get('objects')]
    except:
        print(d['img'], 'not found')
        d['tags'] = None
    

marina (4).jpg not found
tree (2).jpg not found


In [ ]:
for i in ['marina (4).jpg','tree (2).jpg']:
    reduce_image_size(i)

In [53]:
with open(data_folder + '/image_data_complete.json', 'w') as outfile:
    json.dump(data1, outfile)

In [ ]:
# OLD: get image tags via scraping 
driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()))

rec_url = 'https://imagerecognize.com/'
driver.maximize_window()
driver.get(rec_url)

short_wait_time = 3
loading_wait_time = 30
offset = 6
image_tags_dict = {}
for i, image in enumerate(sorted_images[offset:]):
    print(image,'({0}/{1})'.format(i,len(sorted_images[offset:])))
    full_image = os.path.join(absolute_path,image)
    file_drop = driver.find_element(By.ID,"drop_file_zone")
    try:
        file_drop.click()
    except ElementClickInterceptedException as e:
        print('click intercepted, but fuck it we ball')
        file_drop.click()
    time.sleep(1)
    # upload image and enter
    pyautogui.write(full_image)
    pyautogui.press('enter')
    # wait for loading then wait for results to populate
    try:
        WebDriverWait(driver, short_wait_time).until(EC.presence_of_element_located((By.ID,'loading')))
        print('started loading...')
    except:
        print('incorrect file type?')
        continue
    try:
        WebDriverWait(driver, loading_wait_time).until(EC.presence_of_element_located((By.XPATH,'//*[@id="response"]/h3[1]')))
    except:
        print('image too big?')
        continue
    response = driver.find_element(By.ID,'response')
    # '//*[@id="respoC:\Users\tyler\PersonalProjects\small-data-blog\src\components\BlogPosts\BerkeleyNature\Images\animal (8).jpg
    # nse"]/h3[1]'
    children = response.find_elements(By.XPATH,'./h3')
    tags = [c.text.split(' ')[0] for c in children]
    # print(children)
    # save tags in dataframe
    image_tags_dict[image] = tags
image_tags_dict

In [ ]:
image_tags_dict

In [ ]:
tag_df = pd.DataFrame(columns=['img','tags'],data=image_tags_dict.items())
tag_df

In [ ]:
tag_df['date'] = tag_df['img'].map(image_date_dict)
tag_df

In [ ]:
sorted_images_df = pd.DataFrame(sorted_images)
sorted_images_df.to_csv(os.path.join(data_folder,'sorted_images.csv'), index=False)

In [ ]:
# save to excel file for metadata
tag_df.to_csv(os.path.join(data_folder,'image_metadata_copy.csv'), index=False)

Add date to metadata